In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, Normalizer, QuantileTransformer, MinMaxScaler
from const import *

In [2]:
X_train = pd.read_csv('dataset/NSL-KDD/KDDTrain+.txt', header=None, names=col_names_org)
y_train = X_train.iloc[:,-2]
X_train = X_train.iloc[:,:-2]

X_test = pd.read_csv('dataset/NSL-KDD/KDDTest+.txt', header=None, names=col_names_org)
y_test = X_test.iloc[:,-2]
X_test = X_test.iloc[:,:-2]

In [3]:
# one hot categorical data
X_train_enc = X_train.iloc[:,1:4]
X_test_enc = X_test.iloc[:,1:4]

enc = OneHotEncoder(categories=[X_train.protocol_type.unique(),X_train.service.unique(),X_train.flag.unique()])
enc = enc.fit(X_train_enc)
transformed_features = enc.get_feature_names_out()

X_train_enc = pd.DataFrame(enc.transform(X_train_enc).toarray(), columns=transformed_features)
X_test_enc = pd.DataFrame(enc.transform(X_test_enc).toarray(), columns=transformed_features)

In [4]:
# Scale data
X_train_scale = X_train.drop(columns=features_binary + features_category, axis=1)
X_test_scale = X_test.drop(columns=features_binary + features_category, axis=1)

# scaler = StandardScaler()
# scaler = MinMaxScaler((-1, 1)) # best so far
scaler = Normalizer()
scaler = scaler.fit(X_train_scale)
scaled_features = scaler.get_feature_names_out()

X_train_scale = pd.DataFrame(scaler.transform(X_train_scale), columns=scaled_features)
X_test_scale = pd.DataFrame(scaler.transform(X_test_scale), columns=scaled_features)

In [5]:
# Binary data
X_train_bin = X_train.loc[:, features_binary]
X_test_bin = X_test.loc[:, features_binary]

In [6]:
# Join back to main data
X_train = X_train_bin.join([X_train_scale, X_train_enc])
X_test = X_test_bin.join([X_test_scale, X_test_enc])

In [7]:
def get_key_from_value(value):
    if value == 'normal':
        return 0

    for key, vals in attack_category_dict.items():
        if value in vals:
            return key

y_train = y_train.apply(get_key_from_value)
y_test = y_test.apply(get_key_from_value)

In [8]:
X_train = X_train.join(y_train)
X_test = X_test.join(y_test)

X_train.to_csv('dataset/train.csv', index=False)
X_test.to_csv('dataset/test.csv', index=False)

In [9]:
X_train.to_feather('dataset/train.feather')
X_test.to_feather('dataset/test.feather')